<a href="https://colab.research.google.com/github/harbour22/darkcyan-tools/blob/main/DarkCyan-yolo-trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## General Imports
from google.colab import drive # type: ignore
from IPython.display import clear_output # type: ignore
from pathlib import Path
import json
import shutil
import os

In [3]:
if 'COLAB_RELEASE_TAG' in os.environ:
   COLAB_RELEASE_TAG = os.environ['COLAB_RELEASE_TAG']
   colab = True

In [2]:
!pip install --upgrade git+https://github.com/harbour22/darkcyan-tools.git
clear_output(wait=False)

In [3]:
!pip install ultralytics
clear_output(wait=False)

In [4]:
## App Imports
from darkcyan.classify_data_utilities import get_zipfilename
from darkcyan.constants import GOOGLEDRIVE_DATA_ROOT, \
                               DEFAULT_GOOGLEDRIVE_YOLO_DATA_DIR, \
                               DEFAULT_GOOGLEDRIVE_YOLO_CONFIG_DIR, \
                               DEFAULT_GOOGLEDRIVE_YOLO_TRAINING_OUTPUT_DIR, \
                               DEFAULT_YOLO_TRAINING_CONFIG, \
                               YOLOMODELMAP, \
                               YoloBaseModels, \
                               DataType

Config file /root/.darkcyan/config.json does not exist, creating

Config key local_data_repository not found, attempting defaults

Config key scratch_dir not found, attempting defaults

Config key data_prefix not found, attempting defaults

Config key temp_dir not found, attempting defaults

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
## Load config
config_file = Path(GOOGLEDRIVE_DATA_ROOT) / \
                   DEFAULT_GOOGLEDRIVE_YOLO_CONFIG_DIR / \
                   DEFAULT_YOLO_TRAINING_CONFIG

with open(config_file, "r", encoding="utf-8") as f:
    config = json.load(f)
config

{'version': '4.1lb',
 'type': 'cls',
 'basemodel': 'large',
 'epochs': 300,
 'imgsz': 224,
 'batchsize': 128,
 'creation_time': '2023-05-03 22:26:44',
 'created_by': 'chris'}

In [21]:
data_type = DataType['cls']
model_size = YoloBaseModels['large']

batch = config['batchsize']
epochs = config['epochs']
project_path = Path(GOOGLEDRIVE_DATA_ROOT) / \
               DEFAULT_GOOGLEDRIVE_YOLO_TRAINING_OUTPUT_DIR / \
               config['type'] / f"darkcyan_{config['version']}" / \
               model_size.name

data_path = f"/content/{config['type']}_training_data"
imgsz = config['imgsz']

In [22]:
zip_filename = Path(GOOGLEDRIVE_DATA_ROOT) / DEFAULT_GOOGLEDRIVE_YOLO_DATA_DIR / (f"{get_zipfilename(config['version'])}.zip")
shutil.unpack_archive(zip_filename,data_path)

In [ ]:
from ultralytics import YOLO

last_run = project_path / 'train' / 'weights' / 'last.pt'
if last_run.exists():
  base_model = last_run
else: 
  base_model = YOLOMODELMAP[data_type][model_size]

print(base_model)

model = YOLO(base_model) # pass any model type
model.train(epochs=epochs, project=project_path, batch=batch, data=data_path, imgsz=imgsz, exist_ok = False)

/content/drive/MyDrive/yolo/training_output/cls/darkcyan_4.1lb/large/train/weights/last.pt


Ultralytics YOLOv8.0.91 🚀 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=classify, mode=train, model=/content/drive/MyDrive/yolo/training_output/cls/darkcyan_4.1lb/large/train/weights/last.pt, data=/content/cls_training_data, epochs=300, patience=50, batch=128, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolo/training_output/cls/darkcyan_4.1lb/large, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=Fa

In [1]:
import os
os.environ

environ{'SHELL': '/bin/bash',
        'NV_LIBCUBLAS_VERSION': '11.11.3.6-1',
        'NVIDIA_VISIBLE_DEVICES': 'all',
        'COLAB_JUPYTER_TRANSPORT': 'ipc',
        'NV_NVML_DEV_VERSION': '11.8.86-1',
        'NV_CUDNN_PACKAGE_NAME': 'libcudnn8',
        'CGROUP_MEMORY_EVENTS': '/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events',
        'NV_LIBNCCL_DEV_PACKAGE': 'libnccl-dev=2.16.2-1+cuda11.8',
        'NV_LIBNCCL_DEV_PACKAGE_VERSION': '2.16.2-1',
        'VM_GCE_METADATA_HOST': '169.254.169.253',
        'HOSTNAME': '35191ff15633',
        'TBE_RUNTIME_ADDR': '172.28.0.1:8011',
        'GCE_METADATA_TIMEOUT': '3',
        'NVIDIA_REQUIRE_CUDA': 'cuda>=11.8 brand=tesla,driver>=450,driver<451 brand=tesla,driver>=470,driver<471 brand=unknown,driver>=470,driver<471 brand=nvidia,driver>=470,driver<471 brand=nvidiartx,driver>=470,driver<471 brand=geforce,driver>=470,driver<471 brand=geforcertx,driver>=470,driver<471 brand=quadro,driver>=470,driver<471 brand=q